# Week2 
---

- optimization methods such as SGD, Momentum, RMSProp and Adam
- random minibatches
- learning rate decay

## Mini Batch

将数据集分成更小的组进行训练

- SGD的问题, 无法利用向量化加速, 噪音大
- Batch Grad的问题, 对于大数据集, 参数更新太慢, 内存要求大
- Mini Batch优势, 解决上上述两个问题, 能加速计算同时参数更新速度快

## 指数加权平均

指数加权平均递推式;
$$V_t = \beta V_{t-1} + (1-\beta)\theta_t \\
\qquad = (1-\beta)\theta_t + \beta (1-\beta)\theta_{t-1} + \beta^2 (1-\beta)\theta_{t-2} +\dots + \beta^{n-1} (1-\beta)\theta_{1}$$

$V_t$的含义是前$\frac{1}{1-\beta}$个数据的平均, 因为$\beta^{\frac{1}{1-\beta}}=\frac{1}{e} \approx 0.3$, 当权重递减权重低于它, 数据不具有代表性, 该递推式能够反映数据的变化趋势.

> 为什么是这种形式的滑动平均? 具有简洁形式的递推式, 存储占用小

## Bias Correction

由于通常$\theta_0=0$的取值, 指数滑动平均有一个启动过程, 这部分曲线不能够很好的拟合实际数据, Bias Correction是通过添加增加一个随t递减的偏置, 使得曲线在早期也能够较好拟合. 修正方法是利用$\tilde{V_t} = \frac{V_t}{1 - \beta_t}$代替$V_t$, 在早期通过该除数恢复$V_t$的比例, 后期当$\beta_t$趋近0该偏置接近1, 不影响拟合.

事实上, 在优化算法中用到指数加权平均的地方, 使用Bias correction并不常见, 梯度的下降可以接受具有一个启动过程.

## Momentum SGD

将指数滑动平均应用到梯度的更新当中, 计算过程如下:

- 计算得到dW, db
- $V^w_t = \beta^w V^w_t + (1-\beta)dW, \quad V^b_t = \beta^b V^b_t + (1-\beta)db$
- $W = W - \alpha V^w_t, \quad b = b - \alpha V^b_t$

动量SGD能够消除震荡方向的梯度, 保持下降方向的梯度, 加速下降, 减小震荡

有的文章中, 会将表达式写成$V_t = \beta V_{t-1} + dW$, 与上述表达式是等价的, 该表达式整体进行了$1-\beta$的缩放, 与上表达式的区别仅在于缩放后两者具有不同的最佳学习率. 但是后者不够直观, 在调整beta时会影响alpha值, 因此并不建议.

## RMSprop

将指数平滑应用到梯度的均方差根的更新当中, 计算过程如下:

- 计算得到dW, db
- $S^w_t = \beta^w S^w_t + (1-\beta)dW^2, \quad S^b_t = \beta^b S^b_t + (1-\beta)db^2$
- $W = W - \alpha \frac{dW}{\sqrt{S^w_t + \epsilon}}, \quad b = b - \alpha \frac{db}{\sqrt{S^b_t + \epsilon}}$

其中$\epsilon$是个小量, 防止零除, 但对计算无影响.

对RMSprop的直观理解, 震荡大的维度, 均方差大, 因此在更新中, 除数大, 更新的幅度小; 均方差小的维度更加稳定, 因此更新的幅度更大

## Adam (Adaptive Moment Estimation)

Adam是Momentum和RMSprop的结合, 计算过程如下:

- 计算得到dW, db
- $V^w_t = \beta_1^w V^w_t + (1-\beta_1)dW, \quad V^b_t = \beta_1^b V^b_t + (1-\beta_1)db$
- $S^w_t = \beta_2^w S^w_t + (1-\beta_2)dW^2, \quad S^b_t = \beta_2^b S^b_t + (1-\beta_2)db^2$
- $W = W - \alpha \frac{V^w_t}{\sqrt{S^w_t + \epsilon}}, \quad b = b - \alpha \frac{V^b_t}{\sqrt{S^b_t + \epsilon}}$

推荐的超参设置: $\beta_1: 0.9, \beta_2:0.999, \epsilon: 10^{-8}$

## 学习率下降

通过学习率下降获得更好的收敛效果.

策略:

$$ \alpha_t = \frac{1}{1 + decay_rate * Epoch}\cdot \alpha_0 \\
\alpha_t = (decay_rate)^{epoch} \cdot \alpha_0$$

加入decay_rate机制不是首先选择的策略, 但是经验上说, 是在后期提高模型效果的重要且有效的策略

### 如何看待收敛到局部最优的问题

高维参数和高维cost计算函数不容易出现局部最优的问题, 常以鞍点的形式出现. 但是更需要关注的问题是参数下降过程的平台部问题, 使用上述优化方法可以有效的快速通过平台部.